In [0]:
%pip install python-dotenv

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.types import (
    ArrayType,
    DoubleType,
    StringType,
    StructType,
    StructField,
    DateType,
)
from delta.tables import DeltaTable
from sedona.spark import *
import random
import geopandas as gpd
import time
import os
from dotenv import load_dotenv
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [0]:
catalog_dev = "`land_auto-gen-kart_dev`"
schema_dev = "dl_bildesegmentering"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
helipads_table = "helipads_silver"
noHelipads_table = "nohelipads"

BASE_PATH = "/Volumes/land_auto-gen-kart_devv/external_dev/static_data/DL_bildesegmentering"
SUBDIR = {"image": "noHelipad_images", "mask": "noHelipad_labels"}

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {noHelipads_table} (
    lokalid STRING,
    ingest_time TIMESTAMP,
    geometry BINARY,
    bbox ARRAY<DOUBLE>,
    bbox_str STRING,
    image_path STRING,
    mask_path STRING,
    image_wms STRING,
    image_status STRING,
    mask_status STRING,
    load_time TIMESTAMP,
    photo_time TIMESTAMP
) USING DELTA
"""
spark.sql(q)

In [0]:
def read_table_to_wkt():
    """
    Leser polygonene fra helikopterplasstabellen og returnerer dem som en GeoDataFrame med WKT geometri.
    """
    df = (
        spark.read.table(helipads_table)
        .withColumn("Polygons", F.expr("ST_GeomFromWKB(Polygons)"))
        .select("lokalid", "Polygons", "ingest_time")
    )
    return df

In [0]:
def make_bbox(df: DataFrame, column_name: str) -> DataFrame:
    """
    Lager en boks rundt polygonen med en fast avstand fra sentrum.
    """
    df = df.withColumn(
        "bbox",
        F.expr(
            f"""
        array(
            ST_XMin({column_name}) + 128,
            ST_YMin({column_name}) + 128,
            ST_XMax({column_name}) + 128,
            ST_YMax({column_name}) + 128
        )
        """
        ),
    )
    df = df.withColumn(
        "bbox_str",
        F.concat_ws(
            ",",
            F.format_string("%.6f", F.col("bbox")[0]),
            F.format_string("%.6f", F.col("bbox")[1]),
            F.format_string("%.6f", F.col("bbox")[2]),
            F.format_string("%.6f", F.col("bbox")[3]),
        ),
    ).drop(column_name)

    df = df.withColumn(
        "bbox_geom",
        F.expr("ST_MakeEnvelope(bbox[0], bbox[1], bbox[2], bbox[3], 25833)"),
    )
    df = df.withColumn("geometry", F.hex(F.expr("ST_AsBinary(bbox_geom)"))).drop(
        "bbox_geom"
    )

    return df

In [0]:
def generate_image_url(bbox_str):
    """
    Genererer en URL for image-bilde basert på bbox_str.
    """
    width, height = [512, 512]
    return (
        f"https://wms.geonorge.no/skwms1/wms.nib?VERSION=1.3.0"
        f"&service=WMS&request=GetMap&Format=image/png&"
        f"GetFeatureInfo=text/plain&CRS=EPSG:25833&Layers=ortofoto&"
        f"BBox={bbox_str}&width={width}&height={height}"
    )


def image_file_exists(id: str) -> str:
    """
    Sjekker om bildet med gitt ID er lastet ned.
    """
    path = f"/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering/noHelipad_images/image_{id}.png"
    return "DOWNLOADED" if os.path.exists(path) else "PENDING"


def mask_file_exists(id: str) -> str:
    """
    Sjekker om masken med gitt ID er generert.
    """
    path = f"/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering/noHelipad_labels/mask_{id}.png"
    return "GENERATED" if os.path.exists(path) else "PENDING"

In [0]:
load_dotenv()
BRUKERID = os.getenv("GEONORGE_BRUKERID")
PASSORD = os.getenv("GEONORGE_PASSORD")


def get_token():
    """
    Henter token fra GeoNorge og returnerer det.
    """
    url = (
        f"https://baat.geonorge.no/skbaatts/req?brukerid={BRUKERID}"
        f"&passord={PASSORD}&tjenesteid=wms.nib&retformat=s"
    )
    raw_token = requests.get(url).text.strip("`")
    return raw_token


token = get_token()
token_start_time = time.time()
token_lifetime = 55 * 60


def refresh_token_if_needed():
    """
    Henter ny token om den gamle er utløpt.
    """
    global token, token_start_time
    if time.time() - token_start_time > token_lifetime:
        print("🔄 Fornyer token...")
        token = get_token()
        token_start_time = time.time()

In [0]:
def get_fotodato(bbox: str, token: str, max_retries=10):
    """
    Henter fotodato for en bbox.
    """
    url = f"https://wms.geonorge.no/skwms1/wms.nib?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetFeatureInfo&CRS=EPSG:25833&BBOX={bbox}&WIDTH=512&HEIGHT=512&LAYERS=ortofoto&QUERY_LAYERS=ortofoto&INFO_FORMAT=text/html&I=256&J=256&TICKET={token}"

    table = None
    field_value = None
    for i in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            time.sleep(1.0)
            soup = BeautifulSoup(response.text, "html.parser")
            table = soup.find("table")
            break
        except Exception as e:
            wait = 2**i
            print(f"⚠️ Feil ved henting av fotodato ({e}), prøver igjen om {wait}s...")
            time.sleep(wait)

    if table:
        rows = table.find_all("tr")
        for row in rows:
            cells = row.find_all("td")
            if len(cells) >= 2 and cells[0].text.strip() == "Fotodato":
                field_value = cells[1].text.strip()
                field_value = datetime.strptime(field_value, "%d.%m.%Y").date()
                return field_value

    return None

In [0]:
def add_ortofoto_date(df: DataFrame, token: str) -> DataFrame:
    """
    Legger til fotodato kolonnen til en DataFrame.
    """
    # Henter bare relevante kolonner
    sample_rows = df.select("lokalid", "bbox_str").collect()

    # Henter fotodato
    bbox_date_pairs = [
        (row["lokalid"], get_fotodato(row["bbox_str"].replace("_", ","), token))
        for row in sample_rows
    ]

    schema = StructType(
        [
            StructField("lokalid", StringType(), True),
            StructField("photo_time", DateType(), True),
        ]
    )

    bbox_date_df = spark.createDataFrame(bbox_date_pairs, schema)
    df_with_date = df.join(bbox_date_df, on="lokalid", how="left")

    return df_with_date

In [0]:
def write_delta_table(sdf: DataFrame, mode: str = "merge") -> None:
    """
    Skriver data til deltatabellen og opdaterer dersom lokalid allerede finnes.
    """
    if mode == "overwrite":
        sdf.write.format("delta").option("mergeSchema", "true").mode(
            "overwrite"
        ).saveAsTable(noHelipads_table)
    else:
        from delta.tables import DeltaTable

        delta_tbl = DeltaTable.forName(spark, noHelipads_table)

        delta_tbl.alias("target").merge(
            sdf.alias("source"), condition="target.lokalid = source.lokalid"
        ).whenMatchedUpdate(
            condition="target.load_time < source.load_time OR target.image_path IS NULL",
            set={col: f"source.{col}" for col in sdf.columns},
        ).whenNotMatchedInsert(
            values={col: f"source.{col}" for col in sdf.columns}
        ).execute()

In [0]:
generate_image_url_udf = F.udf(generate_image_url, StringType())
image_file_exists_udf = F.udf(image_file_exists, StringType())
mask_file_exists_udf = F.udf(mask_file_exists, StringType())

df = read_table_to_wkt()
df = df.withColumn("lokalid", F.reverse(F.col("lokalid")))
df = make_bbox(df, "Polygons")

for dt in ["image", "mask"]:
    sub = SUBDIR[dt]
    df = df.withColumn(
        f"{dt}_path",
        F.concat(F.lit(f"{BASE_PATH}/{sub}/{dt}_"), F.col("lokalid"), F.lit(".png")),
    )
df = (
    df.withColumn("image_wms", generate_image_url_udf("bbox_str"))
    .withColumn("image_status", image_file_exists_udf("lokalid"))
    .withColumn("mask_status", mask_file_exists_udf("lokalid"))
    .withColumn("load_time", F.current_timestamp())
)
df = add_ortofoto_date(df, token)

write_delta_table(df)